In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertModel, BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import f1_score
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read_json(path):
    data = []
    with open(path, 'r',encoding='utf-8') as fp:
        for line in fp:
            data.append(json.loads(line))
    return data

In [ ]:
class BaselineData(Dataset):
    def __init__(self, data, tokenizer, config):
        self.data = data
        self.tokenizer = tokenizer
        self.pad_size = config.pad_size
        self.label2id = config.label2id
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence = '[SEP]'.join([self.data[idx]['hashtag']] + self.data[idx]['comments'])

        input_ids, attention_mask = self.__convert_to_id__(sentence)

        if self.data[idx].get('attitudes'):
            label = self.__convert_label__(self.data[idx]['attitudes'])
            return torch.tensor(input_ids),\
                   torch.tensor(attention_mask),\
                   torch.tensor(label)
        else:
            return torch.tensor(input_ids),\
                   torch.tensor(attention_mask),\

    def __convert_to_id__(self, sentence):
        ids = self.tokenizer.encode_plus(sentence)
        input_ids = self.__padding__(ids['input_ids'])
        attention_mask = self.__padding__(ids['attention_mask'])

        return input_ids, attention_mask
    
    def __convert_label__(self, label):
        onehot_label = [0] * 24
        for i in label:
            onehot_label[self.label2id[i]] = 1
        return onehot_label

    def __padding__(self, sentence):
        sentence = sentence[:self.pad_size]  # 长就截断
        sentence = sentence + [0] * (self.pad_size-len(sentence))  # 短就补充
        return sentence

In [ ]:
class Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.bert = BertModel.from_pretrained(config.PTM)
        self.bert_config = BertConfig.from_pretrained(config.PTM)
        self.fc = nn.Linear(self.bert_config.hidden_size, self.bert_config.hidden_size)
        self.fc1 = nn.Linear(self.bert_config.hidden_size, config.label_num)
        self.act = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.bert(input_ids=x[0], attention_mask=x[1]).pooler_output
        x = self.sigmoid(self.fc1(self.act(self.fc(x))))
        return x

In [ ]:
def train(config, dataset, model, optimizer, valid_dataset):
    max_scores = 0
    for epoch in range(config.epochs):
        with tqdm(total=len(dataset)) as pbar:
            for idx, data in enumerate(dataset):
                x = [data[0].long().to(config.device), data[1].long().to(config.device)]
                y = data[2].float().to(config.device)
                y_hat = model(x)
                loss = F.binary_cross_entropy(y_hat, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.set_postfix({'loss' : '{:.4f}'.format(loss)})
                pbar.update(1)
        scores = valid(config, valid_dataset, model)
        if scores >= max_scores:
            max_scores = scores
            saved_model = model
    return saved_model

def valid(config, dataset, model):
    true = []
    pred = []
    with torch.no_grad():
        for idx, data in enumerate(dataset):
            x = [data[0].long().to(config.device), data[1].long().to(config.device)]
            y = data[2].float().to(config.device).view(-1, 24).tolist()
            y_hat = model(x).view(-1, 24).tolist()
            true.extend(y)
            pred.extend(y_hat)

    pred = [[1 if i>=0.5 else 0 for i in j] for j in pred]

    micro_f1 = f1_score(pred, true, average='micro')
    macro_f1 = f1_score(pred, true, average='macro')
    print('micro_f1: {:.4f}'.format(micro_f1))
    print('micro_f1: {:.4f}'.format(macro_f1))
    return micro_f1*0.4+macro_f1*0.6

def generate_test_result(config, dataset, model):
    with torch.no_grad():
        predict = []
        for idx, data in enumerate(dataset):
            x = [data[0].long().to(config.device), data[1].long().to(config.device)]
            predict.extend(model(x).view(-1, 24).tolist())
    with open('submit', 'w', encoding='utf-8') as f:
        for i in range(len(predict)):
            line = []
            for j in range(len(predict[i])):
                if predict[i][j] >= 0.5:
                    line.append(config.id2label[j])
            f.write(' '.join([str(i)]+line))
            f.write('\n')

In [ ]:
class Config():
    def __init__(self):
        self.pad_size = 500
        self.batch_size = 24
        self.epochs = 15
        self.PTM = 'bert-base-chinese'
        self.label_num = 24
        self.device = 'cuda:0'
        self.lr = 5e-5

        label_dic = ['[微笑]', '[嘻嘻]', '[笑cry]', '[怒]', '[泪]', '[允悲]', '[憧憬]', '[doge]', '[并不简单]', '[思考]', '[费解]', '[吃惊]', '[拜拜]', '[吃瓜]', '[赞]', '[心]', '[伤心]', '[蜡烛]', '[给力]', '[威武]', '[跪了]', '[中国赞]', '[给你小心心]', '[酸]']

        self.id2label = {k: v for k, v in enumerate(label_dic)}  # 用于标签的部分
        self.label2id = {v: k for k, v in enumerate(label_dic)}

In [ ]:
config = Config()
train_data = read_json('/content/drive/My Drive/FYP_Dataset/train.json')
valid_data = read_json('/content/drive/My Drive/FYP_Dataset/valid.json')
test_data = read_json('/content/drive/My Drive/FYP_Dataset/test.json')

tokenizer = BertTokenizer.from_pretrained(config.PTM)

train_dataloader = DataLoader(BaselineData(train_data, tokenizer, config), batch_size=config.batch_size)
valid_dataloader = DataLoader(BaselineData(valid_data, tokenizer, config), batch_size=config.batch_size)
test_dataloader = DataLoader(BaselineData(test_data, tokenizer, config), batch_size=1)

In [ ]:
model = Model(config).to(config.device)
optimizer = torch.optim.AdamW(model.parameters(), config.lr)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
best_model = train(config, train_dataloader, model, optimizer, valid_dataloader)
generate_test_result(config, test_dataloader, best_model)

100%|██████████| 300/300 [03:20<00:00,  1.50it/s, loss=0.2815]


micro_f1: 0.3827
micro_f1: 0.1518


100%|██████████| 300/300 [03:20<00:00,  1.49it/s, loss=0.2315]


micro_f1: 0.4715
micro_f1: 0.2143


100%|██████████| 300/300 [03:21<00:00,  1.49it/s, loss=0.1921]


...

micro_f1: 0.5503
micro_f1: 0.3545


100%|██████████| 300/300 [03:21<00:00,  1.49it/s, loss=0.0064]


micro_f1: 0.5494
micro_f1: 0.3679


100%|██████████| 300/300 [03:21<00:00,  1.49it/s, loss=0.0041]


micro_f1: 0.5442
micro_f1: 0.3591
